<a href="https://colab.research.google.com/github/HiagoAD/GDI2/blob/master/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

## Load datasets


In [2]:
dengue = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2020.csv", encoding ="UTF-8")
emlurb = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2020.csv", encoding ="UTF-8")

dengue19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2019.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2019.csv", encoding ="UTF-8")

dengue18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2018.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2018.csv", encoding ="UTF-8")

dengue17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2017.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2017.csv", encoding ="UTF-8")

dengue16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2016.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2016.csv", encoding ="UTF-8")


b'Skipping line 108: expected 126 fields, saw 127\nSkipping line 1485: expected 126 fields, saw 127\nSkipping line 2584: expected 126 fields, saw 127\nSkipping line 4415: expected 126 fields, saw 127\nSkipping line 5659: expected 126 fields, saw 127\nSkipping line 6469: expected 126 fields, saw 127\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (55,113) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Dados de 2020 - 2016


Iremos fazer os mesmos tratamentos anteriores para os dados de 2019 até 2016.

In [19]:
# Dados Dengue

merge_dengue = dengue.append([dengue19,dengue18,dengue17,dengue16], ignore_index = True)
merge_dengue = merge_dengue[["febre", "mialgia",'vomito', "dt_notificacao",'nu_idade','nausea','cefaleia','dor_costas','diabetes','renal']]
merge_dengue['dt_notificacao'] = pd.to_datetime(merge_dengue['dt_notificacao'])

# Agrupamos por número de casos por dia

#merge_dengue = merge_dengue.groupby([pd.Grouper(key="dt_notificacao", freq="D")]).size().reset_index(name="CASOS_DENGUE")


# Dados Emlurb

merge_emlurb = emlurb.append([emlurb19,emlurb18,emlurb17,emlurb16], ignore_index = True)
merge_emlurb = merge_emlurb[["GRUPOSERVICO_CODIGO", "DATA_DEMANDA"]]
merge_emlurb = merge_emlurb[(merge_emlurb.GRUPOSERVICO_CODIGO == 11)]
merge_emlurb["DATA_DEMANDA"] = pd.to_datetime(merge_emlurb["DATA_DEMANDA"],errors='coerce')

# Agrupamos por número de solicitações por dia

#merge_emlurb = merge_emlurb.groupby([pd.Grouper(key="DATA_DEMANDA", freq="D")]).size().reset_index(name="CHAMAD0S_EMLURB")

#year_filter = merge_emlurb[(merge_emlurb['DATA_DEMANDA'].dt.year == 2018)]


merge_dengue = merge_dengue.dropna()
merge_dengue['febre'] = merge_dengue['febre'].astype('int')
merge_dengue['mialgia'] = merge_dengue['mialgia'].astype('int')
merge_dengue['vomito'] = merge_dengue['vomito'].astype('int')
merge_dengue['nausea'] = merge_dengue['nausea'].astype('int')
merge_dengue['cefaleia'] = merge_dengue['cefaleia'].astype('int')
merge_dengue['dor_costas'] = merge_dengue['dor_costas'].astype('int')
merge_dengue['diabetes'] = merge_dengue['diabetes'].astype('int')
merge_dengue['renal'] = merge_dengue['renal'].astype('int')
merge_dengue = merge_dengue.sort_values('dt_notificacao')
merge_dengue

,febre,mialgia,vomito,dt_notificacao,nu_idade,nausea,cefaleia,dor_costas,diabetes,renal
14367,1,1,2,2016-01-03,4024,2,2,2,2,2
13813,2,2,2,2016-01-03,4059,2,2,2,2,2
13658,1,1,2,2016-01-04,4017,1,1,2,2,2
13595,1,1,2,2016-01-04,4031,2,1,2,2,2
13781,1,1,2,2016-01-04,4054,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...
975,1,1,1,2020-07-16,4008,1,1,2,2,2
1007,1,1,2,2020-07-17,4017,2,1,2,2,2
1005,1,2,2,2020-07-19,4010,2,2,2,2,2
1003,1,1,2,2020-07-20,3001,2,2,2,2,2


## Feature de predição

1.   Número de casos de dengue por dia.
2.   Número de solicitações de drenagem por dia.



## Separação de dados: Treino, Validação e Teste

Texto.

In [4]:
def split_values(df, column_name):
  X = df.drop(columns=[column_name,'dt_notificacao']).values
  y = df[column_name].values

  return X, y

In [5]:
features_results = merge_dengue.drop(columns=['febre','dt_notificacao']).values
labels_results = merge_dengue['febre'].values

feature_train, feature_test, label_train, label_test = train_test_split(features_results, labels_results, test_size=0.2,random_state=1)
feature_train, feature_validation, label_train, label_validation =  train_test_split(feature_train, label_train, test_size=0.25,random_state=1)

In [6]:
train, validation, test = np.split(merge_dengue.sample(frac=1, random_state=42), [int(.6*len(merge_dengue)), int(.8*len(merge_dengue))])

column_name = 'febre'

X_train, y_train = split_values(train, column_name)
X_valid, y_valid = split_values(validation, column_name)
X_test,  y_test  = split_values(test, column_name)

In [7]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(15547, 8)
(5183, 8)
(5183, 8)


## Escolha de Algoritmos

*   Decision Tree
*   Random Forest
*   Naive Bayes
*   MLP
*   KNN



In [8]:
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
naive_bayes   = GaussianNB()
ml_perceptron = MLPClassifier()

decision_tree.fit(X_train, y_train)
y_decision_tree_train = decision_tree.predict(X_train)
y_decision_tree_test  = decision_tree.predict(X_test)

random_forest.fit(X_train, y_train)
y_random_forest_train = random_forest.predict(X_train)
y_random_forest_test  = random_forest.predict(X_test)

naive_bayes.fit(X_train, y_train)
y_naive_bayes_train = naive_bayes.predict(X_train)
y_naive_bayes_test  = naive_bayes.predict(X_test)

ml_perceptron.fit(X_train, y_train)
y_ml_perceptron_train = ml_perceptron.predict(X_train)
y_ml_perceptron_test  = ml_perceptron.predict(X_test)

In [9]:
print("=== Precisão Testes === \n\n")

print("Precisão Decision Tree:", accuracy_score(y_test, y_decision_tree_test))
print("Precisão Random Forest:", accuracy_score(y_test, y_random_forest_test))
print("Precisão Naive Bayes:" , accuracy_score(y_test, y_naive_bayes_test))
print("Precisão MLP: ", accuracy_score(y_test, y_ml_perceptron_test))

print("\n\n=== Precisão Treino === \n\n")

print("Precisão Decision Tree:", accuracy_score(y_train, y_decision_tree_train))
print("Precisão Random Forest:", accuracy_score(y_train, y_random_forest_train))
print("Precisão Naive Bayes:" , accuracy_score(y_train, y_naive_bayes_train))
print("Precisão MLP: ", accuracy_score(y_train, y_ml_perceptron_train))

=== Precisão Testes === 


Precisão Decision Tree: 0.8815357900829636
Precisão Random Forest: 0.8792205286513602
Precisão Naive Bayes: 0.729500289407679
Precisão MLP:  0.1794327609492572


=== Precisão Treino === 


Precisão Decision Tree: 0.8882099440406509
Precisão Random Forest: 0.8881456229497652
Precisão Naive Bayes: 0.7263137582813405
Precisão MLP:  0.17958448575287836


## MLFlow e Hiper-parâmetros

Nessa etapa, adicionaremos o MLFlow e o Optuna para seleção de hiper-parâmetros sobre o conjunto de validação.

In [10]:
!pip install mlflow --quiet
!pip install optuna --quiet

import mlflow
import optuna

     |████████████████████████████████| 14.4 MB 60 kB/s 
     |████████████████████████████████| 146 kB 56.2 MB/s 
     |████████████████████████████████| 636 kB 56.1 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 170 kB 63.7 MB/s 
     |████████████████████████████████| 1.1 MB 56.4 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 302 kB 5.3 MB/s 
     |████████████████████████████████| 80 kB 5.4 MB/s 
     |████████████████████████████████| 111 kB 10.3 MB/s 
     |████████████████████████████████| 141 kB 9.4 MB/s 
     |████████████████████████████████| 49 kB 3.7 MB/s 


### Decision Tree

In [11]:
def objective_decisionTree(trial):
    max_depth = trial.suggest_int("max_depth", 1, 15, 1)
    split     = trial.suggest_int('min_samples_split', 3, 15, 1)
    leaf      = trial.suggest_int('min_samples_leaf', 3, 15, 1)

    decisionTree_selection = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=leaf, min_samples_split=split)
    decisionTree_selection.fit(X_train, y_train)
    
    prediction_validation = decisionTree_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_dt = optuna.create_study(direction="maximize")
study_dt.optimize(objective_decisionTree, n_trials=50)
print(study_dt.best_trial)

[I 2021-08-15 15:05:13,450] A new study created in memory with name: no-name-c32b60f6-80a3-43ad-a5b8-676124c0cf00
[I 2021-08-15 15:05:13,471] Trial 0 finished with value: 0.8844298668724677 and parameters: {'max_depth': 13, 'min_samples_split': 12, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.8844298668724677.
[I 2021-08-15 15:05:13,493] Trial 1 finished with value: 0.882307543893498 and parameters: {'max_depth': 14, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8844298668724677.
[I 2021-08-15 15:05:13,505] Trial 2 finished with value: 0.8722747443565503 and parameters: {'max_depth': 3, 'min_samples_split': 8, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.8844298668724677.
[I 2021-08-15 15:05:13,524] Trial 3 finished with value: 0.8863592513988038 and parameters: {'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 15}. Best is trial 3 with value: 0.8863592513988038.
[I 2021-08-15 15:05:13,543] Trial 4 finished with value: 0.88

FrozenTrial(number=3, values=[0.8863592513988038], datetime_start=datetime.datetime(2021, 8, 15, 15, 5, 13, 508141), datetime_complete=datetime.datetime(2021, 8, 15, 15, 5, 13, 523758), params={'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 15}, distributions={'max_depth': IntUniformDistribution(high=15, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=15, low=3, step=1), 'min_samples_leaf': IntUniformDistribution(high=15, low=3, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE, value=None)


In [12]:
print(study_dt.best_trial.value)
print(study_dt.best_trial.params)

0.8863592513988038
{'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 15}


### Random Forest

In [13]:
def objective_randomForest(trial):
    forrest_n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    forrest_criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    forrest_max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])

    randomForrest_selection = RandomForestClassifier(n_estimators=forrest_n_estimators, criterion=forrest_criterion, max_features=forrest_max_features)
    randomForrest_selection.fit(X_train, y_train) 

    prediction_validation = randomForrest_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_randomForest, n_trials=50)
print(study_rf.best_trial)

[I 2021-08-15 15:05:14,778] A new study created in memory with name: no-name-814404f9-e2fb-4446-991e-683e97eac712
[I 2021-08-15 15:05:20,132] Trial 0 finished with value: 0.8809569747250627 and parameters: {'n_estimators': 521, 'criterion': 'entropy', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8809569747250627.
[I 2021-08-15 15:05:21,885] Trial 1 finished with value: 0.8803781593671619 and parameters: {'n_estimators': 185, 'criterion': 'gini', 'max_features': 'log2'}. Best is trial 0 with value: 0.8809569747250627.
[I 2021-08-15 15:05:25,806] Trial 2 finished with value: 0.8803781593671619 and parameters: {'n_estimators': 373, 'criterion': 'entropy', 'max_features': 'log2'}. Best is trial 0 with value: 0.8809569747250627.
[I 2021-08-15 15:05:32,483] Trial 3 finished with value: 0.8811499131776963 and parameters: {'n_estimators': 736, 'criterion': 'gini', 'max_features': 'auto'}. Best is trial 3 with value: 0.8811499131776963.
[I 2021-08-15 15:05:40,364] Trial 4 finished wit

FrozenTrial(number=45, values=[0.8819216669882307], datetime_start=datetime.datetime(2021, 8, 15, 15, 9, 53, 744981), datetime_complete=datetime.datetime(2021, 8, 15, 15, 10, 2, 427372), params={'n_estimators': 951, 'criterion': 'gini', 'max_features': 'auto'}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=100, step=1), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=45, state=TrialState.COMPLETE, value=None)


In [14]:
print(study_rf.best_trial.value)
print(study_rf.best_trial.params)

0.8819216669882307
{'n_estimators': 951, 'criterion': 'gini', 'max_features': 'auto'}


### Naive Bayes

In [15]:
def objective_naiveBayes(trial):
    nb_var_smoothing = trial.suggest_float('var_smoothing', 1e-9, 1e9, log=True)

    naiveBayes_selection = GaussianNB(var_smoothing=nb_var_smoothing)
    naiveBayes_selection.fit(X_train, y_train)     

    prediction_validation = naiveBayes_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_nb = optuna.create_study(direction="maximize")
study_nb.optimize(objective_naiveBayes, n_trials=50)
print(study_nb.best_trial)

[I 2021-08-15 15:10:34,886] A new study created in memory with name: no-name-958153f0-3fed-4fa3-85d7-cf86cb8c4208
[I 2021-08-15 15:10:34,904] Trial 0 finished with value: 0.819988423692842 and parameters: {'var_smoothing': 306.9631182608007}. Best is trial 0 with value: 0.819988423692842.
[I 2021-08-15 15:10:34,918] Trial 1 finished with value: 0.819988423692842 and parameters: {'var_smoothing': 56512985.99715345}. Best is trial 0 with value: 0.819988423692842.
[I 2021-08-15 15:10:34,928] Trial 2 finished with value: 0.8143932085664672 and parameters: {'var_smoothing': 6.69732922040443e-05}. Best is trial 0 with value: 0.819988423692842.
[I 2021-08-15 15:10:34,938] Trial 3 finished with value: 0.8143932085664672 and parameters: {'var_smoothing': 0.0002454127560356398}. Best is trial 0 with value: 0.819988423692842.
[I 2021-08-15 15:10:34,949] Trial 4 finished with value: 0.819988423692842 and parameters: {'var_smoothing': 1569952.360597796}. Best is trial 0 with value: 0.81998842369284

FrozenTrial(number=29, values=[0.8730464981670847], datetime_start=datetime.datetime(2021, 8, 15, 15, 10, 35, 237413), datetime_complete=datetime.datetime(2021, 8, 15, 15, 10, 35, 248210), params={'var_smoothing': 1.8753756875332642e-07}, distributions={'var_smoothing': LogUniformDistribution(high=1000000000.0, low=1e-09)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=29, state=TrialState.COMPLETE, value=None)


In [16]:
print(study_nb.best_trial.value)
print(study_nb.best_trial.params)

0.8730464981670847
{'var_smoothing': 1.8753756875332642e-07}


### MLP

In [17]:
def objective_MLP(trial):

    #mlp_selection = MLPClassifier()
    #mlp_selection.fit(X_train, y_train)     

    #prediction_validation = mlp_selection.predict(X_valid)
    #accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_mlp = optuna.create_study(direction="maximize")
study_mlp.optimize(objective_MLP, n_trials=50)
print(study_mlp.best_trial)

[I 2021-08-15 15:10:35,579] A new study created in memory with name: no-name-160e30ba-9e37-4f6d-804a-8dfadb355072
[W 2021-08-15 15:10:35,584] Trial 0 failed because of the following error: NameError("name 'accuracy_validation' is not defined")
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-17-30c8ea155a76>", line 9, in objective_MLP
    return accuracy_validation
NameError: name 'accuracy_validation' is not defined


NameError: ignored

In [ ]:
print(study_mlp.best_trial.value)
print(study_mlp.best_trial.params)

## Diagnóstico de melhor modelo

Após os testes realizados com o Random Forest, Decision Tree, Naive Bayes e MLP, encontramos que o algoritmo que melhor resultado obeteve foi blabla.


In [ ]:
## heatmap

## Feature importance

Verificamos o nível de relevância das colunas do dataset.

# **MLP**

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers, optimizers, metrics, activations

resolver = tensorflow.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tensorflow.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tensorflow.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tensorflow.config.list_logical_devices('TPU'))

In [ ]:
def MLP():
    nu_shape = input_shape[0] * input_shape[1]
    curve = activations.sigmoid
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape),
            layers.Reshape((nu_shape,)),
            layers.Dense(nu_shape, activation=curve),
            layers.Dense(nu_shape, activation=curve),
            layers.Dropout(0.1),
            layers.Dense(nu_shape, activation=curve),
            layers.Dense(num_classes, activation=activations.softmax)
        ]
    )
    model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(learning_rate=1), metrics=["accuracy", metrics.Recall(), metrics.Precision()], steps_per_execution=50)

    return model

In [ ]:
def LSTM(hidden_layer_size, look_back = 7):
  model = Sequential()
  model.add(LSTM(hidden_layer_size, input_shape=(look_back,1)))
  model.add(Dense(1)) # to have a regressor
  model.compile(loss='mean_squared_error', optimizer='adam', steps_per_execution=50)
  return model